## Detection histories summary for Tropical Screech-Owl & Ferruginous Pygmy-Owl

In [1]:
import pandas as pd
import os
import re

In [2]:
# Load the paths
main_dir = "/mnt/d/night_recordings_analysis/"
output_file_fepowl = os.path.join(main_dir, "Ferruginous Pygmy-Owl/fepowl_detection_history.csv")
output_file_trsowl = os.path.join(main_dir, "Tropical_Screech-Owl/detection_history_matrix.csv")

In [3]:
# Load detection history matrix
df_fepowl = pd.read_csv(output_file_fepowl, index_col=0)
df_trsowl = pd.read_csv(output_file_trsowl, index_col=0)

### Count the number of sites with at least one detection

In [5]:
# Convert all values to numeric (just in case)
df_numeric_fepowl = df_fepowl.apply(pd.to_numeric, errors='coerce')
df_numeric_trsowl = df_trsowl.apply(pd.to_numeric, errors='coerce')


# Sum rows → any row (site) with at least one 1
detections_per_site_fepowl = (df_numeric_fepowl == 1).any(axis=1)
num_sites_with_detections_fepowl = detections_per_site_fepowl.sum()

detections_per_site_trsowl = (df_numeric_trsowl == 1).any(axis=1)
num_sites_with_detections_trsowl = detections_per_site_trsowl.sum()

print(f"Number of sites with at least one detection for Ferruginous Pygmy-Owl: {num_sites_with_detections_fepowl}")
print(f"Number of sites with at least one detection for Tropical Screech-Owl: {num_sites_with_detections_trsowl}")

Number of sites with at least one detection for Ferruginous Pygmy-Owl: 17
Number of sites with at least one detection for Tropical Screech-Owl: 53


### Partition detecions across habitat types

In [6]:
# Add habitat type column
# Extract habitat type: CH, M, or P (case-insensitive)
habitat_types_fepowl = df_numeric_fepowl.index.to_series().str.extract(r'^([a-zA-Z]+)', expand=False).str.upper() #Gets letters from the start of each site name

habitat_types_trsowl = df_numeric_trsowl.index.to_series().str.extract(r'^([a-zA-Z]+)', expand=False).str.upper()

# Add this as a column
df_numeric_fepowl['Habitat'] = habitat_types_fepowl
df_numeric_trsowl['Habitat'] = habitat_types_trsowl

In [9]:
# Boolean: True for sites with at least one detection
has_detection_fepowl = (df_numeric_fepowl.drop(columns='Habitat') == 1).any(axis=1)
has_detection_trsowl = (df_numeric_trsowl.drop(columns='Habitat') == 1).any(axis=1)

# Combine into a DataFrame
df_numeric_fepowl['Detected'] = has_detection_fepowl
df_numeric_trsowl['Detected'] = has_detection_trsowl

# Group by habitat and sum the True values
summary_by_habitat_fepowl = df_numeric_fepowl.groupby('Habitat')['Detected'].sum()
summary_by_habitat_trsowl = df_numeric_trsowl.groupby('Habitat')['Detected'].sum()

print("Number of sites with ≥1 detection by habitat: Ferruginous Pygmy-Owl")
print(summary_by_habitat_fepowl)


print("Number of sites with ≥1 detection by habitat: Tropical Screech-Owl")
print(summary_by_habitat_trsowl)


Number of sites with ≥1 detection by habitat: Ferruginous Pygmy-Owl
Habitat
CH     3
M     11
P      3
Name: Detected, dtype: int64
Number of sites with ≥1 detection by habitat: Tropical Screech-Owl
Habitat
CH    25
M     10
P     18
Name: Detected, dtype: int64


In [11]:
# Combine habitat-wise summaries into one DataFrame
summary_combined = pd.DataFrame({
    "Ferruginous Pygmy-Owl": summary_by_habitat_fepowl,
    "Tropical Screech-Owl": summary_by_habitat_trsowl,
}).fillna(0).astype(int)

summary_combined.index.name = "Habitat"

summary_combined.loc["Total"] = summary_combined.sum()

from IPython.display import display

display(summary_combined)

,Ferruginous Pygmy-Owl,Tropical Screech-Owl
Habitat,,
CH,3,25
M,11,10
P,3,18
Total,17,53


In [13]:
#main_dir = "/mnt/d/night_recordings_analysis/"
summary_combined.to_csv(os.path.join(main_dir,"detections_summary.csv"))